In [1]:

# evaluate the performance of distinguisher,compute the tpr,tnr
from AES_128_batch import generate_train_data
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import load_model

gpus = tf.config.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu,enable = True) 

# model_file = "./good_trained_nets/AES128_data_index_None_best_model_2r_depth5_num_epochs20_acc_1.0.h5"
# model_file = "./good_trained_nets/AES128_data_index_None_best_model_3r_depth5_num_epochs20_acc_0.5303980112075806.h5"
model_file = "./3_round_good_trained_nets/AES16_data_index_[0, 13]_best_model_3r_depth5_num_epochs20_acc_0.5042799711227417.h5"
net = load_model(model_file)

nr = 3
index = [0,13]

def evaluate(net,X,Y):
    strategy = tf.distribute.MirroredStrategy(
        # devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3", "/gpu:4"])
        devices=["/gpu:4"])
    print('Number of devices: %d' % strategy.num_replicas_in_sync)  # 输出设备数量
    batch_size = 2000 * strategy.num_replicas_in_sync
    with strategy.scope():
        Z = net.predict(X,batch_size=batch_size).flatten();
    Zbin = (Z > 0.5);
    diff = Y - Z; mse = np.mean(diff*diff);
    n = len(Z); n0 = np.sum(Y==0); n1 = np.sum(Y==1);
    acc = np.sum(Zbin == Y) / n;
    tpr = np.sum(Zbin[Y==1]) / n1;
    tnr = np.sum(Zbin[Y==0] == 0) / n0;
    mreal = np.median(Z[Y==1]);
    high_random = np.sum(Z[Y==0] > mreal) / n0;
    print("np.sum(Zbin[Y==1]) = ",np.sum(Zbin[Y==1]))
    print("n1 = ",n1)
    print("np.sum(Zbin[Y==0] == 0) = ",np.sum(Zbin[Y==0] == 0))
    print("n0 = ",n0)
    print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr, "MSE:", mse);
    print("Percentage of random pairs with score higher than median of real pairs:", 100*high_random);

X, Y = generate_train_data(10**6, nr,data_index=index,flag=False)
# print("X shape = ", X.shape)
evaluate(net, X, Y);


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:4',)
Number of devices: 1
np.sum(Zbin[Y==1]) =  496045
n1 =  500083
np.sum(Zbin[Y==0] == 0) =  7967
n0 =  499917
Accuracy:  0.504012 TPR:  0.9919253403934947 TNR:  0.015936645483150203 MSE: 0.24902146
Percentage of random pairs with score higher than median of real pairs: 49.60723480097696


In [3]:
from math import log2
log2(2**16*3*2*8)

21.584962500721158